In [1]:
import numpy as np

In [19]:
def gen_samples(a,b, n=1000):
    return (b-a)* np.random.random_sample(n) + a

In [3]:
key_ordering = ['10^9 As', 'ns', 'H0', 'w0', 'ombh2', 'omch2', 'nu_mass_ev']


In [4]:
#Table 1. of aemulusnu paper
bounds = [[1.77, 2.43],
         [.93, 1.01],
         [59.5, 74.5],
         [-1.28, -0.72],
         [0.0198, 0.0248],
         [0.11, 0.13],
         [0.01, 0.5]]

In [5]:
random_cosmos = np.array([gen_samples(a,b) for (a,b) in bounds]).T

In [6]:
from aemulusnu_massfunction import emulator_training


In [7]:
emulator = emulator_training.MassFuncAemulusNu_GP_emulator_training()

In [8]:
import torch

In [9]:
random_params = []
for curr_cosmo_values in random_cosmos:
    X = emulator.in_scaler.transform(np.array([curr_cosmo_values]))

    if(tuple(curr_cosmo_values) not in emulator.ComputedParams):
        with torch.no_grad():#, gpytorch.settings.fast_pred_var():
            predictions = emulator.model(torch.from_numpy(X).float())
            mean = predictions.mean.numpy()
        emulator.ComputedParams[tuple(curr_cosmo_values)] = dict(zip(emulator.param_names, mean[0]))

    random_params +=[list(emulator.ComputedParams[tuple(curr_cosmo_values)].values())]
random_params = np.array(random_params)

In [10]:
random_params.shape

(1000, 8)

In [12]:
np.savetxt('/home/users/delon/aemulusnu_hmf_lib/aemulusnu_hmf/data/random_tier2_output.txt', random_params)

In [13]:
np.savetxt('/home/users/delon/aemulusnu_hmf_lib/aemulusnu_hmf/data/random_tier2_input.txt', random_cosmos)

In [14]:
Ms = np.logspace(12, 16, 1000)

In [15]:
from aemulusnu_hmf.massfunction import cosmology

In [17]:
from tqdm import tqdm

In [20]:
random_cosmos = np.array([gen_samples(a,b, n=10) for (a,b) in bounds]).T

In [21]:
random_dn_dM = []
for curr_cosmo_values in tqdm(random_cosmos):
    curr_cosmology = cosmology(dict(zip(key_ordering, curr_cosmo_values)))
    curr_vals = []
    for a in np.linspace(0.33, 1, 100):
        curr_vals += [emulator(curr_cosmology, Ms, a)]
    random_dn_dM += [curr_vals]
random_dn_dM = np.array(random_dn_dM)

100%|██████████| 10/10 [04:40<00:00, 28.00s/it]


In [22]:
random_dn_dM.shape

(10, 100, 1000)

In [26]:
np.reshape(random_dn_dM, (10, 100 * 1000))

array([[2.25455863e-15, 2.20737154e-15, 2.16114061e-15, ...,
        9.57616629e-30, 8.38725082e-30, 7.33979880e-30],
       [2.08134021e-15, 2.03904828e-15, 1.99759179e-15, ...,
        3.65625534e-29, 3.23998230e-29, 2.86896792e-29],
       [1.72340486e-15, 1.68756246e-15, 1.65244319e-15, ...,
        2.51152296e-30, 2.19149502e-30, 1.91063612e-30],
       ...,
       [1.97910409e-15, 1.93782264e-15, 1.89737577e-15, ...,
        4.55448031e-29, 4.03834310e-29, 3.57804699e-29],
       [2.05886775e-15, 2.01813018e-15, 1.97817932e-15, ...,
        1.33117812e-28, 1.18982520e-28, 1.06275072e-28],
       [2.53924742e-15, 2.48858204e-15, 2.43890094e-15, ...,
        1.02968159e-28, 9.17055891e-29, 8.16165888e-29]])

In [27]:
np.savetxt('/home/users/delon/aemulusnu_hmf_lib/aemulusnu_hmf/data/random_tier2_dn_dM_output.txt', 
           np.reshape(random_dn_dM, (10, 100 * 1000)))

In [28]:
np.savetxt('/home/users/delon/aemulusnu_hmf_lib/aemulusnu_hmf/data/random_tier2_dn_dM_input.txt', 
           random_cosmos)